In [1]:
#Project 3 - Notebook 3 (h2o AutoML)

In [2]:
#Dependencies
import numpy as np
import os
import psycopg2
import pandas as pd
import json
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [3]:
import h2o

In [4]:
#Cloud DB Config
SQL_USERNAME = "postgres" #your personal username goes here
SQL_PASSWORD = "airlineproject" #your personal password goes here
DB_NAME = "airline_db" #The default name of the DB
DB_HOST = "airline-db.c0dqcvhs1y58.us-east-2.rds.amazonaws.com"
PORT = 5432
database_path = f"postgresql://{SQL_USERNAME}:{SQL_PASSWORD}@{DB_HOST}:{PORT}/{DB_NAME}"

In [5]:
from sqlalchemy import create_engine
engine = create_engine(database_path)

In [6]:
#main_df = pd.read_sql_query('select "Mkt_Ccode", "Arr_Delay_Time_Actual" from main_airline;', con=engine)
info_df = pd.read_sql_query('select * from main_airline FETCH FIRST 5 ROWS ONLY;', con=engine)

In [7]:
test_df = pd.read_sql_query('select * from main_airline order by random() limit 10000;', con=engine)

In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 32 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   index                                   10000 non-null  int64 
 1   Mkt_Ccode                               10000 non-null  object
 2   Mkt_Fl_No                               10000 non-null  object
 3   Dep_Airport                             10000 non-null  object
 4   Arr_Airport                             10000 non-null  object
 5   Date                                    10000 non-null  int64 
 6   Day_Week                                10000 non-null  object
 7   Sched_Dep_Time_OAG                      10000 non-null  int64 
 8   Gate_Dep_Time_Actual                    10000 non-null  int64 
 9   Sched_Arr_Time_OAG                      10000 non-null  int64 
 10  Gate_Arr_Time_Actual                    10000 non-null  int64 
 11  Sch

In [9]:
test = test_df.drop(columns = ["index", "Gate_Dep_Time_Actual", "Arr_Delay_Time_Actual", "Dep_Delay_Time_Actual", "Gate_Arr_Time_Actual"])

In [10]:
test.head()

,Mkt_Ccode,Mkt_Fl_No,Dep_Airport,Arr_Airport,Date,Day_Week,Sched_Dep_Time_OAG,Sched_Arr_Time_OAG,Sched_Elapsed_Time_CRS,Actual_Gate_Gate_Time,...,Mins_Late_Delay_Code_G_NAS,Mins_Late_Delay_Code_H_Security,Mins_Late_Delay_Code_I_Late_Arr_Flight,First_Gate_Dep_Time,Total_Ground_Away_From_Gate,Longest_Ground_Time_Away_From_Gate,No_Landings_At_Div_Airport,month,Arr_Delay_Time_Actual_Flag,Sched_Dep_Time_OAG_Block
0,UA,6153.0,MCI,IAD,20180810,fri,810,1127,137,142,...,0,0,0,0,0,0,0,aug,0,morning
1,AA,494.0,CLT,RDU,20181023,tue,2246,2339,53,49,...,0,0,0,0,0,0,0,oct,0,late_evening
2,AA,4981.0,ALB,PHL,20180824,fri,1731,1846,75,63,...,0,0,0,0,0,0,0,aug,0,late_afternoon
3,DL,2766,SLC,SAN,20190629,sat,1710,1804,114,108,...,0,0,5,0,0,0,0,jun,1,late_afternoon
4,DL,1481.0,LIT,ATL,20180919,wed,600,838,98,86,...,0,0,0,0,0,0,0,sep,0,morning


In [11]:
train_df = pd.read_sql_query('select * from main_airline order by random() limit 100000;', con=engine)

In [12]:
train = train_df.drop(columns = ["index", "Gate_Dep_Time_Actual", "Arr_Delay_Time_Actual", "Dep_Delay_Time_Actual", "Gate_Arr_Time_Actual"])


In [13]:
test.to_csv('test.csv', index=False)
train.to_csv('train.csv', index=False)

In [14]:
import h2o
from h2o.automl import H2OAutoML

In [15]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.301-b09, mixed mode)
  Starting server from C:\Users\dlken\anaconda3\envs\PythonData\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\dlken\AppData\Local\Temp\tmphup6tz6g
  JVM stdout: C:\Users\dlken\AppData\Local\Temp\tmphup6tz6g\h2o_dlken_started_from_python.out
  JVM stderr: C:\Users\dlken\AppData\Local\Temp\tmphup6tz6g\h2o_dlken_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,10 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.6
H2O_cluster_version_age:,2 days
H2O_cluster_name:,H2O_from_python_dlken_ii9j5k
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.754 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [16]:
train = h2o.import_file("train.csv")
test = h2o.import_file("test.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [17]:
x = train.columns
y = "Arr_Delay_Time_Actual_Flag"
x.remove(y)

In [18]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [19]:
aml = H2OAutoML(max_runtime_secs = 100, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |
18:26:16.479: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████| 100%


In [20]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20210822_182616,0.930451,0.265557,0.922005,0.12481,0.27846,0.0775398
StackedEnsemble_BestOfFamily_AutoML_20210822_182616,0.930328,0.26584,0.921914,0.125077,0.278527,0.0775774
GBM_grid__1_AutoML_20210822_182616_model_1,0.927767,0.276198,0.920002,0.126411,0.280735,0.0788124
DeepLearning_1_AutoML_20210822_182616,0.922025,0.294481,0.910502,0.137351,0.29259,0.0856089
GBM_4_AutoML_20210822_182616,0.91983,0.424999,0.910442,0.129202,0.35415,0.125422
GBM_3_AutoML_20210822_182616,0.918092,0.419925,0.912787,0.127738,0.351471,0.123532
DeepLearning_grid__1_AutoML_20210822_182616_model_1,0.917068,0.302244,0.905266,0.144201,0.300113,0.0900678
GBM_2_AutoML_20210822_182616,0.916043,0.413735,0.906776,0.127073,0.34747,0.120735
GBM_5_AutoML_20210822_182616,0.913934,0.478796,0.905127,0.132261,0.386169,0.149127
GBM_1_AutoML_20210822_182616,0.898108,0.46875,0.893384,0.129014,0.38114,0.145268


In [21]:
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20210822_182616

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.07041430260388923
RMSE: 0.26535693434295105
LogLoss: 0.2452254097912083
Null degrees of freedom: 9957
Residual degrees of freedom: 9952
Null deviance: 12813.438514608366
Residual deviance: 4883.9092614017045
AIC: 4895.9092614017045
AUC: 0.9394816062208254
AUCPR: 0.9317521890467827
Gini: 0.8789632124416509

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5028269074386197: 


,,0,1,Error,Rate
0,0,6363.0,174.0,0.0266,(174.0/6537.0)
1,1,697.0,2724.0,0.2037,(697.0/3421.0)
2,Total,7060.0,2898.0,0.0875,(871.0/9958.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.502827,0.862162,152.0
1,max f2,0.176090,0.849983,262.0
2,max f0point5,0.694525,0.921907,105.0
3,max accuracy,0.572064,0.913537,134.0
4,max precision,0.999942,1.000000,0.0
5,max recall,0.028543,1.000000,389.0
6,max specificity,0.999942,1.000000,0.0
7,max absolute_mcc,0.572064,0.808033,134.0
8,max min_per_class_accuracy,0.199175,0.869044,251.0
9,max mean_per_class_accuracy,0.381672,0.887883,184.0



Gains/Lift Table: Avg response rate: 34.35 %, avg score: 35.44 %


...

,,0,1,Error,Rate
0,0,62650.0,2722.0,0.0416,(2722.0/65372.0)
1,1,7202.0,27426.0,0.208,(7202.0/34628.0)
2,Total,69852.0,30148.0,0.0992,(9924.0/100000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.412139,0.846795,181.0
1,max f2,0.164183,0.837852,271.0
2,max f0point5,0.787385,0.910343,85.0
3,max accuracy,0.524992,0.902050,154.0
4,max precision,0.999916,1.000000,0.0
5,max recall,0.018757,1.000000,393.0
6,max specificity,0.999916,1.000000,0.0
7,max absolute_mcc,0.562640,0.782317,143.0
8,max min_per_class_accuracy,0.177407,0.858265,263.0
9,max mean_per_class_accuracy,0.387724,0.875471,187.0



Gains/Lift Table: Avg response rate: 34.63 %, avg score: 34.63 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.01,0.999986,2.887836,2.887836,1.0000,0.999990,1.000000,0.999990,0.028878,0.028878,188.783643,188.783643,0.028878
1,2,0.02,0.999978,2.887836,2.887836,1.0000,0.999983,1.000000,0.999986,0.028878,0.057757,188.783643,188.783643,0.057757
2,3,0.03,0.999961,2.887836,2.887836,1.0000,0.999970,1.000000,0.999981,0.028878,0.086635,188.783643,188.783643,0.086635
3,4,0.04,0.999939,2.887836,2.887836,1.0000,0.999952,1.000000,0.999974,0.028878,0.115513,188.783643,188.783643,0.115513
4,5,0.05,0.999883,2.887836,2.887836,1.0000,0.999914,1.000000,0.999962,0.028878,0.144392,188.783643,188.783643,0.144392
5,6,0.10,0.998692,2.887259,2.887548,0.9998,0.999448,0.999900,0.999705,0.144363,0.288755,188.725887,188.754765,0.288739
6,7,0.15,0.990142,2.887259,2.887451,0.9998,0.996058,0.999867,0.998489,0.144363,0.433118,188.725887,188.745139,0.433087
7,8,0.20,0.948989,2.868199,2.882638,0.9932,0.971859,0.998200,0.991832,0.143410,0.576528,186.819915,188.263833,0.575977
8,9,0.30,0.421345,2.136710,2.633996,0.7399,0.751036,0.912100,0.911567,0.213671,0.790199,113.671018,163.399561,0.749860
9,10,0.40,0.167663,0.734954,2.159235,0.2545,0.249771,0.747700,0.746118,0.073495,0.863694,-26.504563,115.923530,0.709316
